In [2]:
import os
import psycopg2
import json
import git
import pandas as pd

In [57]:

import os
import json
import pandas as pd

# Set path
path1 = "/Users/mansigour/Desktop/project 1/phonePay/pulse/data/aggregated/transaction/country/india/state/"

# List states
agg_tran_list = os.listdir(path1)

# Prepare columns
columns1 = {
    "States": [], "Years": [], "Quarter": [],
    "Transaction_type": [], "Transaction_count": [], "Transaction_amount": []
}

# Traverse folder structure
for state in agg_tran_list:
    cur_states = os.path.join(path1, state)
    if not os.path.isdir(cur_states):
        continue

    for year in os.listdir(cur_states):
        cur_years = os.path.join(cur_states, year)
        if not os.path.isdir(cur_years):
            continue

        for file in os.listdir(cur_years):
            if not file.endswith(".json"):
                continue

            cur_file_path = os.path.join(cur_years, file)

            with open(cur_file_path, "r") as f:
                B = json.load(f)

            # Some JSON files may not have "transactionData"
            if B["data"] and B["data"].get("transactionData"):
                for i in B["data"]["transactionData"]:
                    name = i["name"]
                    count = i["paymentInstruments"][0]["count"]
                    amount = i["paymentInstruments"][0]["amount"]
                    columns1["Transaction_type"].append(name)
                    columns1["Transaction_count"].append(count)
                    columns1["Transaction_amount"].append(amount)
                    columns1["States"].append(state)
                    columns1["Years"].append(year)
                    columns1["Quarter"].append(int(file.strip(".json")))

# Create DataFrame
aggre_transaction = pd.DataFrame(columns1)

# Clean state names
aggre_transaction["States"] = aggre_transaction["States"].str.replace("andaman-&-nicobar-islands", "Andaman & Nicobar")
aggre_transaction["States"] = aggre_transaction["States"].str.replace("-", " ")
aggre_transaction["States"] = aggre_transaction["States"].str.title()
aggre_transaction['States'] = aggre_transaction['States'].str.replace(
    "Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu"
)

# ✅ Display output
print(aggre_transaction.head(5034))  # Show first few rows

# Optional: Save to CSV
# aggre_transaction.to_csv("aggregated_transaction_data.csv", index=False)

print(aggre_transaction.shape)  # Show first few rows

                 States Years  Quarter          Transaction_type  \
0     Andaman & Nicobar  2022        1     Peer-to-peer payments   
1     Andaman & Nicobar  2022        1         Merchant payments   
2     Andaman & Nicobar  2022        1  Recharge & bill payments   
3     Andaman & Nicobar  2022        1        Financial Services   
4     Andaman & Nicobar  2022        1                    Others   
...                 ...   ...      ...                       ...   
5029           Nagaland  2018        4  Recharge & bill payments   
5030           Nagaland  2018        4     Peer-to-peer payments   
5031           Nagaland  2018        4         Merchant payments   
5032           Nagaland  2018        4        Financial Services   
5033           Nagaland  2018        4                    Others   

      Transaction_count  Transaction_amount  
0                342649        1.682329e+09  
1                284298        3.700916e+08  
2                163401        7.130013e+07  

In [85]:
#aggre_user
path2 = "/Users/mansigour/Desktop/project 1/phonePay/pulse/data/aggregated/user/country/india/state/"
         
agg_user_list = os.listdir(path2)

columns2 = {"States":[], "Years":[], "Quarter":[], "Brands":[],"Transaction_count":[], "Percentage":[]}

for state in agg_user_list:
    cur_states = path2+state+"/"
    agg_year_list = os.listdir(cur_states)
    
    for year in agg_year_list:
        cur_years = cur_states+year+"/"
        agg_file_list = os.listdir(cur_years)
        
        for file in agg_file_list:
            cur_files = cur_years+file
            data = open(cur_files,"r")
            C = json.load(data)

            try:

                for i in C["data"]["usersByDevice"]:
                    brand = i["brand"]
                    count = i["count"]
                    percentage = i["percentage"]
                    columns2["Brands"].append(brand)
                    columns2["Transaction_count"].append(count)
                    columns2["Percentage"].append(percentage)
                    columns2["States"].append(state)
                    columns2["Years"].append(year)
                    columns2["Quarter"].append(int(file.strip(".json")))
            
            except:
                pass

aggre_user = pd.DataFrame(columns2)

aggre_user["States"] = aggre_user["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
aggre_user["States"] = aggre_user["States"].str.replace("-"," ")
aggre_user["States"] = aggre_user["States"].str.title()
aggre_user['States'] = aggre_user['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")



print(aggre_user.head(6732))
print(aggre_user.shape)


                 States Years  Quarter   Brands  Transaction_count  Percentage
0     Andaman & Nicobar  2022        1     Vivo              18688    0.244966
1     Andaman & Nicobar  2022        1   Xiaomi              17957    0.235384
2     Andaman & Nicobar  2022        1  Samsung              12955    0.169817
3     Andaman & Nicobar  2022        1   Realme               6769    0.088730
4     Andaman & Nicobar  2022        1     Oppo               6461    0.084692
...                 ...   ...      ...      ...                ...         ...
6727           Nagaland  2018        4    Apple               1148    0.026700
6728           Nagaland  2018        4   Lenovo               1024    0.023816
6729           Nagaland  2018        4   Huawei                849    0.019746
6730           Nagaland  2018        4     Asus                840    0.019536
6731           Nagaland  2018        4   Others               6249    0.145336

[6732 rows x 6 columns]
(6732, 6)


In [110]:
#map_transaction
path3 = "/Users/mansigour/Desktop/project 1/phonePay/pulse/data/map/transaction/hover/country/india/state/"
map_tran_list = os.listdir(path3)

columns3 = {"States":[], "Years":[], "Quarter":[],"District":[], "Transaction_count":[],"Transaction_amount":[]}

for state in map_tran_list:
    cur_states = path3+state+"/"
    map_year_list = os.listdir(cur_states)
    
    for year in map_year_list:
        cur_years = cur_states+year+"/"
        map_file_list = os.listdir(cur_years)
        
        for file in map_file_list:
            cur_files = cur_years+file
            data = open(cur_files,"r")
            E = json.load(data)

            for i in E['data']["hoverDataList"]:
                name = i["name"]
                count = i["metric"][0]["count"]
                amount = i["metric"][0]["amount"]
                columns3["District"].append(name)
                columns3["Transaction_count"].append(count)
                columns3["Transaction_amount"].append(amount)
                columns3["States"].append(state)
                columns3["Years"].append(year)
                columns3["Quarter"].append(int(file.strip(".json")))

map_transaction = pd.DataFrame(columns3)

map_transaction["States"] = map_transaction["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
map_transaction["States"] = map_transaction["States"].str.replace("-"," ")
map_transaction["States"] = map_transaction["States"].str.title()
map_transaction['States'] = map_transaction['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")

print(map_transaction.head(20604))
print(map_transaction.shape)


                  States Years  Quarter                           District  \
0      Andaman & Nicobar  2022        1  north and middle andaman district   
1      Andaman & Nicobar  2022        1             south andaman district   
2      Andaman & Nicobar  2022        1                  nicobars district   
3      Andaman & Nicobar  2022        2  north and middle andaman district   
4      Andaman & Nicobar  2022        2             south andaman district   
...                  ...   ...      ...                                ...   
20599           Nagaland  2018        4                  tuensang district   
20600           Nagaland  2018        4                     peren district   
20601           Nagaland  2018        4                   dimapur district   
20602           Nagaland  2018        4                 zunheboto district   
20603           Nagaland  2018        4                     wokha district   

       Transaction_count  Transaction_amount  
0               

In [125]:

#map_user
path4 = "/Users/mansigour/Desktop/project 1/phonePay/pulse/data/map/user/hover/country/india/state/"
map_user_list = os.listdir(path4)

columns4 = {"States":[], "Years":[], "Quarter":[], "Districts":[], "RegisteredUser":[], "AppOpens":[]}

for state in map_user_list:
    cur_states = path4+state+"/"
    map_year_list = os.listdir(cur_states)
    
    for year in map_year_list:
        cur_years = cur_states+year+"/"
        map_file_list = os.listdir(cur_years)
        
        for file in map_file_list:
            cur_files = cur_years+file
            data = open(cur_files,"r")
            F = json.load(data)

            for i in F["data"]["hoverData"].items():
                district = i[0]
                registereduser = i[1]["registeredUsers"]
                appopens = i[1]["appOpens"]
                columns4["Districts"].append(district)
                columns4["RegisteredUser"].append(registereduser)
                columns4["AppOpens"].append(appopens)
                columns4["States"].append(state)
                columns4["Years"].append(year)
                columns4["Quarter"].append(int(file.strip(".json")))

map_user = pd.DataFrame(columns4)

map_user["States"] = map_user["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
map_user["States"] = map_user["States"].str.replace("-"," ")
map_user["States"] = map_user["States"].str.title()
map_user['States'] = map_user['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")


print(map_user.head(20608))
print(map_user.shape)   


                  States Years  Quarter                          Districts  \
0      Andaman & Nicobar  2022        1  north and middle andaman district   
1      Andaman & Nicobar  2022        1             south andaman district   
2      Andaman & Nicobar  2022        1                  nicobars district   
3      Andaman & Nicobar  2022        2  north and middle andaman district   
4      Andaman & Nicobar  2022        2             south andaman district   
...                  ...   ...      ...                                ...   
20603           Nagaland  2018        4                  tuensang district   
20604           Nagaland  2018        4                     peren district   
20605           Nagaland  2018        4                   dimapur district   
20606           Nagaland  2018        4                 zunheboto district   
20607           Nagaland  2018        4                     wokha district   

       RegisteredUser  AppOpens  
0               10720    9048

In [152]:
#top_transaction
path5 =  "/Users/mansigour/Desktop/project 1/phonePay/pulse/data/top/transaction/country/india/state/"

top_tran_list = os.listdir(path5)

columns5 = {"States":[], "Years":[], "Quarter":[], "Pincodes":[], "Transaction_count":[], "Transaction_amount":[]}

for state in top_tran_list:
    cur_states = path5+state+"/"
    top_year_list = os.listdir(cur_states)
    
    for year in top_year_list:
        cur_years = cur_states+year+"/"
        top_file_list = os.listdir(cur_years)
        
        for file in top_file_list:
            cur_files = cur_years+file
            data = open(cur_files,"r")
            H = json.load(data)

            for i in H["data"]["pincodes"]:
                entityName = i["entityName"]
                count = i["metric"]["count"]
                amount = i["metric"]["amount"]
                columns5["Pincodes"].append(entityName)
                columns5["Transaction_count"].append(count)
                columns5["Transaction_amount"].append(amount)
                columns5["States"].append(state)
                columns5["Years"].append(year)
                columns5["Quarter"].append(int(file.strip(".json")))

top_transaction = pd.DataFrame(columns5)

top_transaction["States"] = top_transaction["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
top_transaction["States"] = top_transaction["States"].str.replace("-"," ")
top_transaction["States"] = top_transaction["States"].str.title()
top_transaction['States'] = top_transaction['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")

print(top_transaction.head(9999))
print(top_transaction.shape)  


                 States Years  Quarter Pincodes  Transaction_count  \
0     Andaman & Nicobar  2022        1   744101             209941   
1     Andaman & Nicobar  2022        1   744103             157230   
2     Andaman & Nicobar  2022        1   744105             122716   
3     Andaman & Nicobar  2022        1   744102              74883   
4     Andaman & Nicobar  2022        1   744104              32471   
...                 ...   ...      ...      ...                ...   
9994           Nagaland  2018        4   797113               5861   
9995           Nagaland  2018        4   798601               5806   
9996           Nagaland  2018        4   797117               4640   
9997           Nagaland  2018        4   797118               4531   
9998           Nagaland  2018        4   797116               4309   

      Transaction_amount  
0           6.159781e+08  
1           3.997160e+08  
2           3.218506e+08  
3           1.933365e+08  
4           7.291552e+07

In [153]:
#top_user
path6 = "/Users/mansigour/Desktop/project 1/phonePay/pulse/data/top/user/country/india/state/"
top_user_list = os.listdir(path6)

columns6 = {"States":[], "Years":[], "Quarter":[], "Pincodes":[], "RegisteredUser":[]}

for state in top_user_list:
    cur_states = path6+state+"/"
    top_year_list = os.listdir(cur_states)

    for year in top_year_list:
        cur_years = cur_states+year+"/"
        top_file_list = os.listdir(cur_years)

        for file in top_file_list:
            cur_files = cur_years+file
            data = open(cur_files,"r")
            I = json.load(data)

            for i in I["data"]["pincodes"]:
                name = i["name"]
                registeredusers = i["registeredUsers"]
                columns6["Pincodes"].append(name)
                columns6["RegisteredUser"].append(registereduser)
                columns6["States"].append(state)
                columns6["Years"].append(year)
                columns6["Quarter"].append(int(file.strip(".json")))

top_user = pd.DataFrame(columns6)

top_user["States"] = top_user["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
top_user["States"] = top_user["States"].str.replace("-"," ")
top_user["States"] = top_user["States"].str.title()
top_user['States'] = top_user['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")


print(top_user.head(10000))
print(top_user.shape)  



                 States Years  Quarter Pincodes  RegisteredUser
0     Andaman & Nicobar  2022        1   744103            1391
1     Andaman & Nicobar  2022        1   744105            1391
2     Andaman & Nicobar  2022        1   744101            1391
3     Andaman & Nicobar  2022        1   744102            1391
4     Andaman & Nicobar  2022        1   744202            1391
...                 ...   ...      ...      ...             ...
9995           Nagaland  2018        4   797120            1391
9996           Nagaland  2018        4   798601            1391
9997           Nagaland  2018        4   797117            1391
9998           Nagaland  2018        4   797118            1391
9999           Nagaland  2018        4   797116            1391

[10000 rows x 5 columns]
(10000, 5)


In [159]:
#aggre_insurance
path7= "/Users/mansigour/Desktop/project 1/phonePay/pulse/data/aggregated/insurance/country/india/state/"

agg_insur_list= os.listdir(path7)

columns7= {"States":[], "Years":[], "Quarter":[], "Insurance_type":[], "Insurance_count":[],"Insurance_amount":[] }

for state in agg_insur_list:
    cur_states =path7+state+"/"
    agg_year_list = os.listdir(cur_states)
    
    for year in agg_year_list:
        cur_years = cur_states+year+"/"
        agg_file_list = os.listdir(cur_years)

        for file in agg_file_list:
            cur_files = cur_years+file
            data = open(cur_files,"r")
            A = json.load(data)

            for i in A["data"]["transactionData"]:
                name = i["name"]
                count = i["paymentInstruments"][0]["count"]
                amount = i["paymentInstruments"][0]["amount"]
                columns7["Insurance_type"].append(name)
                columns7["Insurance_count"].append(count)
                columns7["Insurance_amount"].append(amount)
                columns7["States"].append(state)
                columns7["Years"].append(year)
                columns7["Quarter"].append(int(file.strip(".json")))


aggre_insurance = pd.DataFrame(columns7)

aggre_insurance["States"] = aggre_insurance["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
aggre_insurance["States"] = aggre_insurance["States"].str.replace("-"," ")
aggre_insurance["States"] = aggre_insurance["States"].str.title()
aggre_insurance['States'] = aggre_insurance['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")



print(aggre_insurance.head(682))
print(aggre_insurance.shape)  




                States Years  Quarter Insurance_type  Insurance_count  \
0    Andaman & Nicobar  2022        1      Insurance              768   
1    Andaman & Nicobar  2022        2      Insurance              501   
2    Andaman & Nicobar  2022        3      Insurance              643   
3    Andaman & Nicobar  2022        4      Insurance              901   
4    Andaman & Nicobar  2024        1      Insurance             1371   
..                 ...   ...      ...            ...              ...   
677           Nagaland  2021        3      Insurance               91   
678           Nagaland  2021        4      Insurance              183   
679           Nagaland  2020        2      Insurance               41   
680           Nagaland  2020        3      Insurance               89   
681           Nagaland  2020        4      Insurance               62   

     Insurance_amount  
0           1131287.0  
1            779740.0  
2            902466.0  
3           1441210.0  
4  

In [163]:
import os
import json
import pandas as pd

# Correct path
path8 = "/Users/mansigour/Desktop/project 1/phonePay/pulse/data/map/insurance/hover/country/india/state/"
map_insur_list = os.listdir(path8)

columns8 = {
    "States": [], "Years": [], "Quarter": [],
    "Districts": [], "Transaction_count": [], "Transaction_amount": []
}

for state in map_insur_list:
    cur_states = path8 + state + "/"
    if not os.path.isdir(cur_states):  # ✅ Skip .DS_Store or invalid files
        continue

    agg_year_list = os.listdir(cur_states)

    for year in agg_year_list:
        cur_years = cur_states + year + "/"
        if not os.path.isdir(cur_years):
            continue

        agg_file_list = os.listdir(cur_years)

        for file in agg_file_list:
            cur_files = cur_years + file
            with open(cur_files, "r") as data:
                D = json.load(data)

            for i in D["data"]["hoverDataList"]:
                name = i["name"]
                count = i["metric"][0]["count"]
                amount = i["metric"][0]["amount"]
                columns8["Districts"].append(name)
                columns8["Transaction_count"].append(count)
                columns8["Transaction_amount"].append(amount)
                columns8["States"].append(state)
                columns8["Years"].append(year)
                columns8["Quarter"].append(int(file.strip(".json")))

# Create DataFrame
map_insurance = pd.DataFrame(columns8)

# Clean state names
map_insurance["States"] = map_insurance["States"].str.replace("andaman-&-nicobar-islands", "Andaman & Nicobar")
map_insurance["States"] = map_insurance["States"].str.replace("-", " ")
map_insurance["States"] = map_insurance["States"].str.title()
map_insurance["States"] = map_insurance["States"].str.replace(
    "Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu"
)

# Show results
print(map_insurance.head(13876))
print(map_insurance.shape)


                  States Years  Quarter                          Districts  \
0      Andaman & Nicobar  2022        1  north and middle andaman district   
1      Andaman & Nicobar  2022        1             south andaman district   
2      Andaman & Nicobar  2022        1                  nicobars district   
3      Andaman & Nicobar  2022        2  north and middle andaman district   
4      Andaman & Nicobar  2022        2             south andaman district   
...                  ...   ...      ...                                ...   
13871           Nagaland  2020        4                    kohima district   
13872           Nagaland  2020        4                  tuensang district   
13873           Nagaland  2020        4                   dimapur district   
13874           Nagaland  2020        4                 zunheboto district   
13875           Nagaland  2020        4                     wokha district   

       Transaction_count  Transaction_amount  
0               

In [167]:
import os
import json
import pandas as pd

# Define the path
path9 = "/Users/mansigour/Desktop/project 1/phonePay/pulse/data/top/insurance/country/india/state/"
top_insur_list = os.listdir(path9)

# Initialize dictionary to hold data
columns9 = {
    "States": [], "Years": [], "Quarter": [],
    "Pincodes": [], "Transaction_count": [], "Transaction_amount": []
}

# Traverse through state folders
for state in top_insur_list:
    cur_states = path9 + state + "/"
    if not os.path.isdir(cur_states):
        continue  # Skip files like `.DS_Store`

    top_year_list = os.listdir(cur_states)

    for year in top_year_list:
        cur_years = cur_states + year + "/"
        if not os.path.isdir(cur_years):
            continue

        top_file_list = os.listdir(cur_years)

        for file in top_file_list:
            cur_files = cur_years + file
            with open(cur_files, "r") as data:
                G = json.load(data)

            for i in G["data"].get("pincodes", []):  # use `.get()` to avoid KeyError
                entityName = i.get("entityName", None)
                count = i.get("metric", {}).get("count", 0)
                amount = i.get("metric", {}).get("amount", 0)

                columns9["Pincodes"].append(entityName)
                columns9["Transaction_count"].append(count)
                columns9["Transaction_amount"].append(amount)
                columns9["States"].append(state)
                columns9["Years"].append(year)
                columns9["Quarter"].append(int(file.strip(".json")))

# Create DataFrame
top_insur = pd.DataFrame(columns9)

# Clean state names
top_insur["States"] = top_insur["States"].str.replace("andaman-&-nicobar-islands", "Andaman & Nicobar")
top_insur["States"] = top_insur["States"].str.replace("-", " ")
top_insur["States"] = top_insur["States"].str.title()
top_insur["States"] = top_insur["States"].str.replace(
    "Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu"
)

# Preview
print(top_insur.head(6668))
print(top_insur.shape)



                 States Years  Quarter Pincodes  Transaction_count  \
0     Andaman & Nicobar  2022        1   744103                174   
1     Andaman & Nicobar  2022        1   744101                149   
2     Andaman & Nicobar  2022        1   744105                137   
3     Andaman & Nicobar  2022        1   744102                102   
4     Andaman & Nicobar  2022        1   744112                 36   
...                 ...   ...      ...      ...                ...   
6663           Nagaland  2020        4   797113                  4   
6664           Nagaland  2020        4   798620                  3   
6665           Nagaland  2020        4   797120                  3   
6666           Nagaland  2020        4   798621                  2   
6667           Nagaland  2020        4   797111                  2   

      Transaction_amount  
0               226181.0  
1               204646.0  
2               218321.0  
3               147402.0  
4                39197.0

In [172]:
#Table Creation
#pgsql connection
mydb = psycopg2.connect(host = "localhost",
                        user = "postgres",
                        password = "Lgsharma12345@",
                        database = "phonepe_data",
                        port = "5432"
                        )
cursor = mydb.cursor()

#aggregated insurance table
create_query7= '''CREATE TABLE if not exists aggregated_insurance (States varchar(50),
                                                                      Years int,
                                                                      Quarter int,
                                                                      Insurance_type varchar(50),
                                                                      Insurance_count bigint,
                                                                      Insurance_amount bigint
                                                                      )'''
cursor.execute(create_query7)
mydb.commit()

for index,row in aggre_insurance.iterrows():
    insert_query7 = '''INSERT INTO aggregated_insurance (States, Years, Quarter, Insurance_type, Insurance_count, Insurance_amount)
                                                        values(%s,%s,%s,%s,%s,%s)'''
    values = (row["States"],
              row["Years"],
              row["Quarter"],
              row["Insurance_type"],
              row["Insurance_count"],
              row["Insurance_amount"]
              )
    cursor.execute(insert_query7,values)
    mydb.commit()

#aggregated transaction table
create_query1 = '''CREATE TABLE if not exists aggregated_transaction (States varchar(50),
                                                                      Years int,
                                                                      Quarter int,
                                                                      Transaction_type varchar(50),
                                                                      Transaction_count bigint,
                                                                      Transaction_amount bigint
                                                                      )'''
cursor.execute(create_query1)
mydb.commit()

for index,row in aggre_transaction.iterrows():
    insert_query1 = '''INSERT INTO aggregated_transaction (States, Years, Quarter, Transaction_type, Transaction_count, Transaction_amount)
                                                        values(%s,%s,%s,%s,%s,%s)'''
    values = (row["States"],
              row["Years"],
              row["Quarter"],
              row["Transaction_type"],
              row["Transaction_count"],
              row["Transaction_amount"]
              )
    cursor.execute(insert_query1,values)
    mydb.commit()

#aggregated user table
create_query2 = '''CREATE TABLE if not exists aggregated_user (States varchar(50),
                                                                Years int,
                                                                Quarter int,
                                                                Brands varchar(50),
                                                                Transaction_count bigint,
                                                                Percentage float)'''
cursor.execute(create_query2)
mydb.commit()

for index,row in aggre_user.iterrows():
    insert_query2 = '''INSERT INTO aggregated_user (States, Years, Quarter, Brands, Transaction_count, Percentage)
                                                    values(%s,%s,%s,%s,%s,%s)'''
    values = (row["States"],
              row["Years"],
              row["Quarter"],
              row["Brands"],
              row["Transaction_count"],
              row["Percentage"])
    cursor.execute(insert_query2,values)
    mydb.commit()

#map_insurance_table
create_query8 = '''CREATE TABLE if not exists map_insurance (States varchar(50),
                                                                Years int,
                                                                Quarter int,
                                                                District varchar(50),
                                                                Transaction_count bigint,
                                                                Transaction_amount float)'''
cursor.execute(create_query8)
mydb.commit()

for index,row in map_insurance.iterrows():
            insert_query8 = '''
                INSERT INTO map_insurance (States, Years, Quarter, District, Transaction_count, Transaction_amount)
                VALUES (%s, %s, %s, %s, %s, %s)

            '''
            values = (
                row['States'],
                row['Years'],
                row['Quarter'],
                row['Districts'],
                row['Transaction_count'],
                row['Transaction_amount']
            )
            cursor.execute(insert_query8,values)
            mydb.commit() 

#map_transaction_table
create_query3 = '''CREATE TABLE if not exists map_transaction (States varchar(50),
                                                                Years int,
                                                                Quarter int,
                                                                District varchar(50),
                                                                Transaction_count bigint,
                                                                Transaction_amount float)'''
cursor.execute(create_query3)
mydb.commit()

for index,row in map_transaction.iterrows():
            insert_query3 = '''
                INSERT INTO map_Transaction (States, Years, Quarter, District, Transaction_count, Transaction_amount)
                VALUES (%s, %s, %s, %s, %s, %s)

            '''
            values = (
                row['States'],
                row['Years'],
                row['Quarter'],
                row['District'],
                row['Transaction_count'],
                row['Transaction_amount']
            )
            cursor.execute(insert_query3,values)
            mydb.commit() 


#map_user_table
create_query4 = '''CREATE TABLE if not exists map_user (States varchar(50),
                                                        Years int,
                                                        Quarter int,
                                                        Districts varchar(50),
                                                        RegisteredUser bigint,
                                                        AppOpens bigint)'''
cursor.execute(create_query4)
mydb.commit()

for index,row in map_user.iterrows():
    insert_query4 = '''INSERT INTO map_user (States, Years, Quarter, Districts, RegisteredUser, AppOpens)
                        values(%s,%s,%s,%s,%s,%s)'''
    values = (row["States"],
              row["Years"],
              row["Quarter"],
              row["Districts"],
              row["RegisteredUser"],
              row["AppOpens"])
    cursor.execute(insert_query4,values)
    mydb.commit()

#top_insurance_table
create_query9 = '''CREATE TABLE if not exists top_insurance (States varchar(50),
                                                                Years int,
                                                                Quarter int,
                                                                Pincodes int,
                                                                Transaction_count bigint,
                                                                Transaction_amount bigint)'''
cursor.execute(create_query9)
mydb.commit()

for index,row in top_insur.iterrows():
    insert_query9 = '''INSERT INTO top_insurance (States, Years, Quarter, Pincodes, Transaction_count, Transaction_amount)
                                                    values(%s,%s,%s,%s,%s,%s)'''
    values = (row["States"],
              row["Years"],
              row["Quarter"],
              row["Pincodes"],
              row["Transaction_count"],
              row["Transaction_amount"])
    cursor.execute(insert_query9,values)
    mydb.commit()

#top_transaction_table
create_query5 = '''CREATE TABLE if not exists top_transaction (States varchar(50),
                                                                Years int,
                                                                Quarter int,
                                                                pincodes int,
                                                                Transaction_count bigint,
                                                                Transaction_amount bigint)'''
cursor.execute(create_query5)
mydb.commit()

for index,row in top_transaction.iterrows():
    insert_query5 = '''INSERT INTO top_transaction (States, Years, Quarter, Pincodes, Transaction_count, Transaction_amount)
                                                    values(%s,%s,%s,%s,%s,%s)'''
    values = (row["States"],
              row["Years"],
              row["Quarter"],
              row["Pincodes"],
              row["Transaction_count"],
              row["Transaction_amount"])
    cursor.execute(insert_query5,values)
    mydb.commit()

#top_user_table
create_query6 = '''CREATE TABLE if not exists top_user (States varchar(50),
                                                        Years int,
                                                        Quarter int,
                                                        Pincodes int,
                                                        RegisteredUser bigint
                                                        )'''
cursor.execute(create_query6)
mydb.commit()

for index,row in top_user.iterrows():
    insert_query6 = '''INSERT INTO top_user (States, Years, Quarter, Pincodes, RegisteredUser)
                                            values(%s,%s,%s,%s,%s)'''
    values = (row["States"],
              row["Years"],
              row["Quarter"],
              row["Pincodes"],
              row["RegisteredUser"])
    cursor.execute(insert_query6,values)
    mydb.commit()